In [ ]:
import os
import pandas as pd

# Set root directory paths for AWS
root_dir = "/var/www/html/separation_t4_t10/top3_and_top10_data"
output_dir = "/var/www/html/separation_t4_t10/Part 1 - Exclusive"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to process a city
def process_city(city_path, city_name):
    """
    Process a single city to extract rank 4-10 results by removing top 3 results from top 10 data.
    
    Args:
        city_path (str): Path to the city directory
        city_name (str): Name of the city
    
    Returns:
        int: Number of exclusive rank 4-10 results found
    """
    try:
        # Build file paths
        top3_path = os.path.join(city_path, "top_3_results_filtered.csv")
        top10_path = os.path.join(city_path, "top_10_results_filtered.csv")

        # Check if both files exist
        if not (os.path.exists(top3_path) and os.path.exists(top10_path)):
            print(f"Skipping {city_name}: Missing required files")
            return 0

        # Read CSV files
        try:
            df_top3 = pd.read_csv(top3_path)
            df_top10 = pd.read_csv(top10_path)
        except Exception as e:
            print(f"Skipping {city_name}: Error reading files - {e}")
            return 0

        # Check for required columns
        required_columns = ['link', 'keyword']
        if not all(col in df_top3.columns for col in required_columns) or not all(col in df_top10.columns for col in required_columns):
            print(f"Skipping {city_name}: Missing required columns")
            return 0

        # Create city output directory
        city_output_dir = os.path.join(output_dir, city_name)
        os.makedirs(city_output_dir, exist_ok=True)

        # Get all unique keywords for this city
        keywords = set(df_top3['keyword'].unique()) | set(df_top10['keyword'].unique())

        # Store all 4-10 results from all keywords
        all_exclusive_rows = []
        keyword_stats = {}

        # Process each keyword
        for keyword in keywords:
            # Filter data for this keyword
            top3_keyword = df_top3[df_top3['keyword'] == keyword]
            top10_keyword = df_top10[df_top10['keyword'] == keyword]

            # Get unique links from top3
            top3_links = set(top3_keyword['link'].dropna())

            # Remove top3 links from top10 to get rank 4-10 results
            mask = ~top10_keyword['link'].isin(top3_links)
            df_exclusive = top10_keyword[mask].copy()

            # If there are remaining results, collect them
            if not df_exclusive.empty:
                # Add a new column to identify these as 4-10 rank results
                df_exclusive['rank_range'] = '4-10'

                # Collect results
                all_exclusive_rows.append(df_exclusive)

                # Record statistics for this keyword
                keyword_stats[keyword] = {
                    'top3_count': len(top3_keyword),
                    'top10_count': len(top10_keyword),
                    'exclusive_count': len(df_exclusive)
                }

        # Combine and save results
        if all_exclusive_rows:
            # Combine all keyword results into one dataframe
            combined_df = pd.concat(all_exclusive_rows, ignore_index=True)
            combined_path = os.path.join(city_output_dir, f"{city_name}_top_4_to_10_results.csv")
            combined_df.to_csv(combined_path, index=False)

            # Create statistics file
            stats_df = pd.DataFrame.from_dict(keyword_stats, orient='index')
            stats_df.reset_index(inplace=True)
            stats_df.rename(columns={'index': 'keyword'}, inplace=True)
            stats_path = os.path.join(city_output_dir, f"{city_name}_stats.csv")
            stats_df.to_csv(stats_path, index=False)

            print(f"Processed {city_name}: Aggregated {len(combined_df)} rank 4-10 results from {len(keyword_stats)} keywords")
            return len(combined_df)
        else:
            print(f"Skipping {city_name}: No rank 4-10 results found")
            return 0

    except Exception as e:
        print(f"Skipping {city_name}: Processing error - {e}")
        return 0

def main():
    """
    Main function to process all cities and extract rank 4-10 results.
    """
    # Check if root directory exists
    if not os.path.exists(root_dir):
        print(f"Error: Root directory {root_dir} does not exist")
        return

    # Statistics variables
    total_exclusive_links = 0
    processed_cities = 0
    skipped_cities = []

    # Get all city directories
    all_cities = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    print(f"Starting to process {len(all_cities)} cities...")

    # Process each city
    for dir_name in all_cities:
        city_path = os.path.join(root_dir, dir_name)

        # Process the city
        exclusive_count = process_city(city_path, dir_name)

        if exclusive_count > 0:
            total_exclusive_links += exclusive_count
            processed_cities += 1
        else:
            skipped_cities.append(dir_name)

    # Print summary results
    print("\n----- Results Summary -----")
    print(f"Successfully processed: {processed_cities}/{len(all_cities)} cities")
    print(f"Skipped cities: {len(skipped_cities)}")
    if skipped_cities:
        print(f"Skipped city list: {', '.join(skipped_cities[:10])}{'...' if len(skipped_cities) > 10 else ''}")
    print(f"Total rank 4-10 links: {total_exclusive_links}")
    print(f"Average links per city: {total_exclusive_links / processed_cities if processed_cities > 0 else 0:.2f}")

    # Save overall summary to file
    summary_data = {
        'total_cities': len(all_cities),
        'processed_cities': processed_cities,
        'skipped_cities': len(skipped_cities),
        'total_exclusive_links': total_exclusive_links,
        'average_links_per_city': total_exclusive_links / processed_cities if processed_cities > 0 else 0
    }
    
    summary_df = pd.DataFrame([summary_data])
    summary_path = os.path.join(output_dir, "processing_summary.csv")
    try:
        summary_df.to_csv(summary_path, index=False)
        print(f"\nProcessing summary saved to: {summary_path}")
    except Exception as e:
        print(f"Error saving summary: {e}")

# Execute main() when this script is run directly
if __name__ == "__main__":
    main()